# Working with the ScienceImage Class [v2]
    v2 -- Updated for new ScienceImage class by JFH
    vX -- THIS IS DEFUNCT

In [1]:
# imports
import os
import numpy as np
from importlib import reload

In [2]:
# Path to PYPIT-Development-suite
pypdev_path = os.getenv('PYPEIT_DEV')

----

# Development

In [5]:
from pypeit.core import procimg
from pypeit import biasframe
from pypeit import flatfield
from pypeit import processimages
from pypeit import scienceimage
from pypeit import traceslits
from pypeit import waveimage
from pypeit import wavetilts

## Settings

In [ ]:
settings = dict(masters={})
settings['masters']['directory'] = pypdev_path+'/Cooked/MF_shane_kast_blue'
settings['masters']['reuse'] = True
settings['masters']['loaded'] = []
#
setup = 'A_01_aa'
#
settings['combine'] = {}

In [ ]:
spectrograph = 'shane_kast_blue'
det = 1

In [ ]:
settings['detector'] = {}
settings['detector']['num'] = det
settings['detector']['dataext'] = 0
settings['detector']['datasec01'] = [[0, 1024], [0, 0]]
settings['detector']['datasec02'] = [[1024, 2048], [0, 0]]
settings['detector']['oscansec01'] = [[2049, 2080], [0, 0]]
settings['detector']['oscansec02'] = [[2080, 2111], [0, 0]]
settings['detector']['naxis0'] = 2112  # Raw frame, with overscan
settings['detector']['naxis1'] = 350
settings['detector']['numamplifiers'] = 2
settings['detector']['gain'] = [1.2, 1.2]
settings['detector']['ronoise'] = [3.7, 3.7]
settings['detector']['saturation'] = 65535.
settings['detector']['nonlinear'] = 0.76
settings['detector']['dispaxis'] = 1
settings['detector']['darkcurr'] = 0.
settings['detector']['binning'] = '1x1'

In [ ]:
settings['trace'] = {}
settings['trace']['object'] = {}
settings['trace']['object']['order'] = 2                # What is the order of the polynomial function to be used to fit the object trace in each slit
settings['trace']['object']['function'] = 'legendre'    # What function should be used to trace the object in each slit? (polynomial, legendre, chebyshev)
settings['trace']['object']['find'] = 'standard'         # What algorithm to use for finding objects [standard, nminima]
settings['trace']['object']['nsmooth'] = 3              # Parameter for Gaussian smoothing when the nminima algorithm is used
settings['trace']['object']['xedge'] = 0.03             # Ignore any objects within xedge of the edge of the slit

In [ ]:
settings['science'] = {}
settings['science']['extraction'] = {}
settings['science']['extraction']['reuse'] = False          # If the science frame has previously been extracted and saved, load the extractions
settings['science']['extraction']['profile'] = 'gaussian'   # Fitting function used to extract science data, only if the extraction is 2D (options are: gaussian, gaussfunc, moffat, moffatfunc) ### NOTE: options with suffix 'func' fits a function to the pixels whereas those without this suffix takes into account the integrated function within each pixel (and is closer to truth)
settings['science']['extraction']['maxnumber'] = 999        # Maximum number of objects to extract in a science frame
settings['science']['extraction']['manual01'] = {}
settings['science']['extraction']['manual01']['frame'] = None
settings['science']['extraction']['manual01']['params'] = None # Info for desired extraction [det,x_pixel_location, y_pixel_location,[x_range,y_range]]


## Load up

In [ ]:
# Bias
settings['bias'] = {}
settings['bias']['useframe'] = 'bias'
#
biasFrame = biasframe.BiasFrame(setup=setup, settings=settings)
msbias = biasFrame.master()
msbias.shape

In [ ]:
# Traceslits
settings['trace']['slits'] = traceslits.default_settings()['trace']['slits'].copy()
traceSlits = traceslits.TraceSlits(None, None, setup=setup, settings=settings)
traceSlits.master()

In [ ]:
# Wavelengths
waveImage = waveimage.WaveImage(setup=setup, settings=settings)
wave = waveImage.master()
wave.shape

In [ ]:
# Tilts
waveTilts = wavetilts.WaveTilts(None, setup=setup, settings=settings)
tilts = waveTilts.master()
tilts.shape

In [ ]:
# Flat
flatField = flatfield.FlatField(settings=settings, setup=setup)
msflat = flatField.master()

In [ ]:
# Maskslits
maskslits = np.array([False])

In [ ]:
# datasec image
reload(arprocimg)
datasec_img, _, _ = arprocimg.get_datasec_trimmed(
    spectrograph, None, settings['detector']['num'],
    settings['detector'],
    naxis0=settings['detector']['naxis0'],
    naxis1=settings['detector']['naxis1'])
datasec_img.shape

In [ ]:
# BPM
bpm = np.zeros_like(datasec_img)

## Instantiate

In [ ]:
files = [pypdev_path+'/RAW_DATA/Shane_Kast_blue/600_4310_d55/b27.fits.gz']

In [ ]:
reload(processimages)
reload(scienceimage)

sciI = scienceimage.ScienceImage(file_list=files,
                                    spectrograph=spectrograph,
                                     settings=settings,
                                     tilts=tilts,
                                     det=det,
                                     tslits_dict=traceSlits.tslits_dict,
                                     pixlocn=traceSlits.pixlocn,
                                     datasec_img=datasec_img,
                                     maskslits=maskslits,
                                     setup=setup,
                                         bpm=bpm
                                    )

## Process

In [ ]:
_ = sciI._process(msbias, msflat, apply_gain=True, dnoise=0.)

### View

In [ ]:
sciI.show('sci')

In [ ]:
sciI.show('rawvar')

In [ ]:
sciI.show('crmasked')

## SkySub

In [ ]:
settings['skysub'] = {}
settings['skysub']['perform']=True
settings['skysub']['method']= 'bspline'
settings['skysub']['bspline'] = {}
settings['skysub']['bspline']['everyn']=20
#
global_sky, modelvar = sciI.global_skysub(settings)

In [ ]:
sciI.show('global')

In [ ]:
sciI.show('image', image=sciI.sciframe-sciI.global_sky)

## Find Object

In [ ]:
settings['trace']

In [ ]:
tracelist, nobj = sciI.find_objects()
nobj

In [ ]:
tracelist[0].keys()

## Another round of skysub

In [ ]:
_ = sciI.global_skysub(settings, use_tracemask=True)

In [ ]:
sciI.show('global')

In [ ]:
sciI.show('skysub')

## Another round of finding objects

In [ ]:
_, nobj = sciI.find_objects()
nobj

## Extraction

In [ ]:
sciI.extraction(wave)